In [3]:
import numpy as np
# Import math functions from NumPy
from numpy import *

In [4]:
def binomialTree(
    So: float = 100, 
    K: float = 100, 
    t: int = 4, 
    option_type: str = "C", 
    E: int = 1, 
    interest_rate: float = 1, 
    vol: float = 0.2, 
    exercise_type: str = "EUR"
    ):
    '''
    Evaluating present option value.
    So: Stock price today.
    K: Strike of the option.
    t: Time steps for expiration.
    option_type: Option type. ("C" for Call and "P" for Put)
    E: Expiration days.
    interest_rate: Interest rate.
    vol: Volatility.
    exercise_type: Type of exercise. ("US" for American and "EUR" for European)
    '''

    Su, Sv = So, So
    
    prices_matrix = np.zeros((t + 1, t + 1))
    option_price_matrix = np.zeros((t + 1, t + 1))
    payoff_matrix = np.zeros((t + 1, t + 1))
    delta_hedge_matrix = np.zeros((t + 1, t + 1))
    
    return_array = []
    
    dt = E / t
    discount_factor = 1 / (1 + interest_rate * dt)
    prob = 0.5 + interest_rate * np.sqrt(dt) / (2 * vol)
    payoff_type = 1 if option_type == "C" else -1
    early_exercise = 1 if exercise_type == "US" else 0
    v = 1 + vol * np.sqrt(dt)
    u = 1 - vol * np.sqrt(dt)

    for col in range(t + 1):
        for idx in range(col + 1):
            prices_matrix[0][col] = Sv
            prices_matrix[col][col] = Su
            payoff_matrix[0][col] = max(payoff_type * (prices_matrix[0][col] - K), 0)
            payoff_matrix[col][col] = max(payoff_type * (prices_matrix[col][col] - K), 0)
            
            if idx < len(prices_matrix[0]) - 1 and col > 1 and idx < col:
                prices_matrix[idx][col] = prices_matrix[idx][col - 1] * v
                prices_matrix[idx + 1][col] = prices_matrix[idx - 1][col - 1] * u
                payoff_matrix[idx][col] = max(payoff_type * (prices_matrix[idx][col] - K), 0)
        Sv = Sv * v
        Su = Su * u
            
    for col in range(t, -1, -1):
        for idx in range(col, -1, -1):
            if col == t:
                option_price_matrix[idx][col] = max(payoff_type * (prices_matrix[idx][col] - K), early_exercise * max(payoff_type * (prices_matrix[idx][col] - K), 0))
            else:
                if idx < t and col < t:
                    option_price_matrix[idx][col] = max(discount_factor * (prob * option_price_matrix[idx][col + 1] + (1 - prob) * option_price_matrix[idx + 1][col + 1]), early_exercise * max(payoff_type * (prices_matrix[idx][col] - K), 0))
                    delta_hedge_matrix[idx][col] = (option_price_matrix[idx][col + 1] - option_price_matrix[idx + 1][col + 1])/(prices_matrix[idx][col + 1] - prices_matrix[idx + 1][col + 1])

    delta_hedge_matrix[:, -1] = 0

    for col in range(t + 1):
        for idx in range(t + 1):
            data_dict = {
                'stock_price': prices_matrix[idx][col],
                'payoff': payoff_matrix[idx][col],
                'option_price': option_price_matrix[idx][col],
                'delta': delta_hedge_matrix[idx][col],
                }
            return_array.append(data_dict)

    return np.reshape(return_array, (len(prices_matrix),len(prices_matrix)))

In [5]:
binomialTree?

Signature:
binomialTree(
    So: float = 100,
    K: float = 100,
    t: int = 4,
    option_type: str = 'C',
    E: int = 1,
    interest_rate: float = 1,
    vol: float = 0.2,
    exercise_type: str = 'EUR',
)
Docstring:
Evaluating present option value.
So: Stock price today.
K: Strike of the option.
t: Time steps for expiration.
option_type: Option type. ("C" for Call and "P" for Put)
E: Expiration days.
interest_rate: Interest rate.
vol: Volatility.
exercise_type: Type of exercise. ("US" for American and "EUR" for European)
File:      c:\users\angelo\appdata\local\temp\ipykernel_14156\1952146775.py
Type:      function


In [14]:
binomialTree(
    So = 100,
    K = 100,
    t = 4,
    option_type = 'C',
    E = 1,
    interest_rate = 1,
    vol = 0.2,
    exercise_type = 'EUR',
)[0][0]

{'stock_price': 100.0,
 'payoff': 0.0,
 'option_price': 47.95963200000006,
 'delta': 0.1613079999999988}

# 2

In [7]:
def binomial_option(spot, strike, rate, sigma, time, steps, output=0):
    
    """
    binomial_option(spot, strike, rate, sigma, time, steps, output=0)
    
    Function for building binomial option tree for european call option payoff. 
    
    Parameters
    ----------
    spot        int or float   - spot price
    strike      int or float   - strike price 
    rate        float          - interest rate
    sigma       float          - volatility
    time        int or float   - expiration time
    steps       int            - number of time steps
    output      int            - [0: price, 1: payoff, 2: option value, 3: option delta]
    
    Returns
    ----------
    out : ndarray
    An array object of price, payoff, option value and delta as specified by the output parameter

    """
    
    # define parameters
    ts = time/steps                                 # ts is time steps, dt
    u = 1 + sigma*sqrt(ts)                          # u is up factor
    v = 1 - sigma*sqrt(ts)                          # v is down factor
    p = 0.5 + rate*sqrt(ts)/(2*sigma)               # p here is risk neutral probability (p') - for ease of use
    df = 1/(1+rate*ts)                              # df is discount factor

    # initialize arrays
    px = zeros((steps+1, steps+1))                  # price path
    cp = zeros((steps+1, steps+1))                  # call intrinsic payoff
    V = zeros((steps+1, steps+1))                   # option value
    d = zeros((steps+1, steps+1))                   # delta value
    
    # binomial loop
    for j in range(steps+1):
        for i in range(j+1):
            px[i,j] = spot * power(v,i) * power(u,j-i)
            cp[i,j] = maximum(px[i,j] - strike,0)
            
    for j in range(steps+1, 0, -1):
        for i in range(j):
            if (j == steps+1):
                V[i,j-1] = cp[i,j-1]                # terminal payoff
                d[i,j-1] = 0                        # terminal delta
            else:
                V[i,j-1] = df*(p*V[i,j]+(1-p)*V[i+1,j])
                d[i,j-1] = (V[i,j]-V[i+1,j])/(px[i,j]-px[i+1,j])
                    
    results = around(px,2), around(cp,2), around(V,2), around(d,4)
    
    return results[output]

In [8]:
px = binomial_option(100,100,0.05,0.2,1,4,0)
px

array([[100.  , 110.  , 121.  , 133.1 , 146.41],
       [  0.  ,  90.  ,  99.  , 108.9 , 119.79],
       [  0.  ,   0.  ,  81.  ,  89.1 ,  98.01],
       [  0.  ,   0.  ,   0.  ,  72.9 ,  80.19],
       [  0.  ,   0.  ,   0.  ,   0.  ,  65.61]])

In [9]:
# Intrinsic value of call options
cp=binomial_option(100,100,0.05,0.2,1,4,1)
cp

array([[ 0.  , 10.  , 21.  , 33.1 , 46.41],
       [ 0.  ,  0.  ,  0.  ,  8.9 , 19.79],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ]])

In [10]:
# Option price 
opx=binomial_option(100,100,0.05,0.2,1,4,2)
opx

array([[10.29, 15.88, 23.83, 34.33, 46.41],
       [ 0.  ,  3.39,  6.11, 10.99, 19.79],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ]])

In [11]:
# Option delta
delta=binomial_option(100,100,0.05,0.2,1,4,3)
delta

array([[0.6241, 0.8053, 0.9645, 1.    , 0.    ],
       [0.    , 0.3393, 0.5553, 0.9086, 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    ]])

In [12]:
# Binomial Option Price
print(f"European Call Option Price using Binomial Tree Method: {opx[0,0]:.2f}")

European Call Option Price using Binomial Tree Method: 10.29
